In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
from ipywidgets import IntProgress

In [2]:
print(tf.__version__)

2.0.0-rc0


In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True)

    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /=255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 150

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validations_targets = next(iter(validation_data))


In [5]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(32,(3, 3), activation='relu', input_shape=(28,28,1)),
                            tf.keras.layers.MaxPooling2D((2,2)),
                            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                            tf.keras.layers.MaxPooling2D((2,2)),
                            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               2

In [7]:
custom_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [8]:
NUM_EPOCHS = 10
NUM_STEPS = num_validation_samples/BATCH_SIZE

early_stopping = tf.keras.callbacks.EarlyStopping(patience=0)


model.fit(train_data,epochs=NUM_EPOCHS,callbacks=[early_stopping], validation_data=(validation_inputs,validations_targets),validation_steps=NUM_STEPS, verbose=2)

Epoch 1/10


360/360 - 19s - loss: 0.2155 - accuracy: 0.9350 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
360/360 - 18s - loss: 0.0526 - accuracy: 0.9842 - val_loss: 0.0517 - val_accuracy: 0.9843
Epoch 3/10
360/360 - 18s - loss: 0.0360 - accuracy: 0.9887 - val_loss: 0.0374 - val_accuracy: 0.9887
Epoch 4/10
360/360 - 18s - loss: 0.0286 - accuracy: 0.9911 - val_loss: 0.0402 - val_accuracy: 0.9878


In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] 0.0355 - accuracy: 0.98 - 2s 2s/step - loss: 0.0355 - accuracy: 0.9878


In [10]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.04. Test accuracy: 98.78%
